# Recurrent Neural Network for MNIST

This notebook implements a (dynamic) multilayer recurrent neural network (RNN) using long short term memory (LSTM) units and TensorFlow. It contains detailed explanations for each step.

In [2]:
import os
import warnings
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mnist import MNIST

% matplotlib inline
warnings.simplefilter('ignore')

## MNIST dataset

The MNIST dataset contains handwritten digits and is available [here](http://yann.lecun.com/exdb/mnist/). To proceed you first have to download the training data and labels and the test data and labels. Next, unpack them into a folder named "mnist" in your home directory. To make the data extraction work, you will have to rename the dots to -, for example t10k-images.idx3-ubyte must be renamed to t10k-images-idx3-ubyte.

To read the MNIST files we will use the [python mnist package](https://pypi.org/project/python-mnist/). You can install the package via

```bash
pip install python-mnist
```

In [3]:
path_to_data = os.path.expanduser("~/mnist/")
mnist = MNIST(path_to_data)

x_train, y_train = mnist.load_training()
x_test, y_test = mnist.load_testing()

In [4]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(f"Training set shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")

print(f"Test set shape: {x_test.shape}")
print(f"Test set shape: {y_test.shape}")

Training set shape: (60000, 784)
Training labels shape: (60000,)
Test set shape: (10000, 784)
Test set shape: (10000,)


## Specifying model and dataset parameters

In [5]:
# Global parameters
eta = 0.01 # learning rate
n_epochs = 4
n_input = 28
n_classes = 10
batch_size = 100
n_train_samples = x_train.shape[0]
n_test_samples = x_test.shape[0]

n_train_batches = n_train_samples // batch_size
n_test_batches = n_test_samples // batch_size

# Network parameters
n_hidden = 50 # number of hidden units per layer
n_layers = 3 # number of layers 
n_steps = 28 # number of truncated backprop steps

## Creating one-hot encoded labels

In [6]:
y_train_one_hot = np.zeros((n_train_samples, n_classes))
y_train_one_hot[np.arange(n_train_samples), y_train] = 1

y_test_one_hot = np.zeros((n_test_samples, n_classes))
y_test_one_hot[np.arange(n_test_samples), y_test] = 1

## Recurrent neural network

### Network architecture

The basic architecture of a recurrent network looks as follows ([source](http://www.deeplearningbook.org/)). 

![title](figures/basic_rnn.png)


As visibile in the figure, the state $h^{(t)}$  of the network depends both on the input $x^{(t)}$ and the previous state $h^{(t-1)}$.
It is computed as follows:

$$ h^{(t)} = \sigma(U x^{(t)} + W h^{(t-1)} + b) $$

with $\sigma$ beign the $\tanh$ in our implementation.


The output is computed as: 

$$ o^{(t)} = V h^{(t)} + c $$
$$ \hat{y}^{(t)} = \text{softmax}(o^{(t)}) $$


### Truncated backpropagation

A recurrent neural network is designed in a way such that the output at a certain time step depends on arbitrarily distant inputs. In other words: when building an RNN in TensorFlow, the graph would have to be as wide as the input sequence.
Unfortunately, this makes backpropagation computation both expensive and ineffective because gradients propagated over many time steps tend to either vanish (most of the time) or explode.

A common solution to this problem is to create an "unrolled" version of the recurrent network that contains a fixed number (*n_steps*) of RNN inputs and outputs. In other words: backpropagation is "truncated" such that errors are only backpropagated for a fixed number of steps. A higher number of steps enables capturing long-term dependencies but is also more expensive (both regarding memory and computation).

The model is then trained on this finite approximation of the recurrent network. Accordingly, at each time step the network is fed with inputs of length *n_steps*. The backward pass is performed after each input block. A short explanation is given on TensorFlow's [website](https://www.tensorflow.org/tutorials/recurrent#truncated-backpropagation).

## Dynamic vs. static RNN 

Tensorflow has two different versions of the RNN functions, namely *tf.nn.static_rnn* and *tf.nn.dynamic_rnn*. 

*tf.nn.static_rnn* creates an unrolled graph for a *fixed* RNN length. For example, when calling *tf.nn.rnn* with an input sequence of length 200, a static graph with 200 time steps is created. This has the disadvantage that we cannot feed longer or shorter sequences into the network than originally specified.

*tf.nn.dynamic_rnn* solves this problem. It dynamically constructs the graph when it's executed, reducing the time requirement for graph creation and allowing for the input batches to vary in size.

One difference between the two functions is the form of the input data. Whereas *tf.nn.static_rnn* takes a list of tensors as an input (namely a list of  n_steps tensors with shape (batch_size, input_size), *tf.nn.dynamic_rnn* takes as input the whole tensor of shape (batch_size, n_steps, input_size).

In [7]:
def make_cell():
    cell = tf.contrib.rnn.LSTMCell(num_units=n_hidden)
    return cell

In [9]:
tf.reset_default_graph()

# Create placeholder variables for the input and targets
X_placeholder = tf.placeholder(tf.float32, shape=[batch_size, n_steps, n_input])
y_placeholder = tf.placeholder(tf.int32, shape=[batch_size, n_classes])

# Create placeholder variables for final weight and bias matrix 
V = tf.Variable(tf.random_normal(shape=[n_hidden, n_classes]))
c = tf.Variable(tf.random_normal(shape=[n_classes]))

# To create multiple layers we call the MultiRNNCell function that takes 
# a list of RNN cells as an input and wraps them into a single cell
cell = tf.contrib.rnn.MultiRNNCell([make_cell() for _ in range(n_layers)], state_is_tuple=True)

# Create a zero-filled state tensor as an initial state
init_state = cell.zero_state(batch_size, tf.float32)

# Create a recurrent neural network specified by "cell", i.e. unroll the
# network.
# Returns a list of all previous RNN hidden states and the final states.
# final_state contains n_layer LSTMStateTuple that contain both the 
# final hidden and the cell state of the respective layer.
outputs, final_state = tf.nn.dynamic_rnn(cell, 
                                         X_placeholder, 
                                         initial_state=init_state)

## Gather final activations

Because we are performing sequence *classification*, we are only interested in the output activations of the last timestep. Since tensorflow does not support negative indexing, we first transpose the tensor such that the "n_steps" axis is first. Then, we use tf.gather to select the correct slice. This process is illustrated below for the following parameter setting: batch_size=100, n_steps=28, n_hidden=20

![title](figures/tensor_transformation.jpg)

In [10]:
temp = tf.transpose(outputs, [1,0,2])
last_output = tf.gather(temp, int(temp.get_shape()[0]-1))

## Network output and loss function

In [11]:
# After gathering the final activations we can easily compute the logits
# using a single matrix multiplication
logits = tf.matmul(last_output, V) + c

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_placeholder,
                                                           logits=logits))

train_step = tf.train.AdamOptimizer(eta).minimize(loss)

## Accuracy

We compute the accuracy of our model as follows.
First, we use *tf.argmax* which gives us the highest entry in a tensor along some axis. For example, *tf.argmax(logits,1)* gives us the label our model considers to be most likely for each input. The true labels are computed using *tf.argmax(y_placeholder,1)*. In a next step, we compare these two tensors using *tf.equal* resulting in a tensor of boolean values.

To compute the accuracy, we first cast the boolean values to floats using *tf.cast*. In a last step, we take the mean of all values.

In [16]:
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_placeholder,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training and testing

We train the network for the specified number of epochs. In each epoch, we run through all training examples, computing the loss and accuracy after 100 batches.

After the training phase is done, we evaluate our model on the test set and report the average loss and accuracy.

In [17]:
with tf.Session() as sess:
    
    # We first have to initialize all variables
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # TRAINING
    for epoch in range(n_epochs):
        
        print()
        print(f"Epoch: {epoch}")
        
        for i in range(n_train_batches):           
            start = i * batch_size
            end = (i + 1) * batch_size

            x_batch = x_train[start:end]        
            x_batch = x_batch.reshape((batch_size, n_steps, n_input))
            y_batch = y_train_one_hot[start:end]
            
            _train_step = sess.run(train_step, 
                                        feed_dict=
                                        {X_placeholder:x_batch,
                                         y_placeholder:y_batch
                                        })
            
            
            if i % 100 == 0:
                _loss, _accuracy = sess.run([loss, accuracy],
                                 feed_dict={
                                     X_placeholder:x_batch,
                                     y_placeholder:y_batch
                                 })
                
                print(f"Minibatch loss: {_loss:.3f}   " 
                      f"Accuracy: {_accuracy:.3f}")
          
    print()
    print(f"Optimization done! Let's calculate the test error")
   
    # TESTING
    losses = []
    acc = []

    for i in range(n_test_batches):

        start = i * batch_size
        end = (i + 1) * batch_size

        x_test_batch = x_test[start:end]        
        x_test_batch = x_test_batch.reshape((batch_size, n_steps, n_input))
        y_test_batch = y_test_one_hot[start:end]


        test_loss, test_accuracy, _train_step = sess.run([loss, accuracy, train_step],
                                                        feed_dict={
                                                            X_placeholder:x_test_batch,
                                                            y_placeholder:y_test_batch
                                                        })

        losses.append(test_loss)
        acc.append(test_accuracy)

    print()
    print(f"Average loss on test set: {np.mean(test_loss):.3f}")
    print(f"Accuracy on test set: {np.mean(test_accuracy):.3f}")


Epoch: 0
Minibatch loss: 2.303   Accuracy: 0.160
Minibatch loss: 0.322   Accuracy: 0.910
Minibatch loss: 0.295   Accuracy: 0.920
Minibatch loss: 0.203   Accuracy: 0.950
Minibatch loss: 0.186   Accuracy: 0.970
Minibatch loss: 0.091   Accuracy: 0.960

Epoch: 1
Minibatch loss: 0.138   Accuracy: 0.960
Minibatch loss: 0.162   Accuracy: 0.970
Minibatch loss: 0.167   Accuracy: 0.940
Minibatch loss: 0.064   Accuracy: 0.970
Minibatch loss: 0.065   Accuracy: 0.990
Minibatch loss: 0.054   Accuracy: 0.980

Epoch: 2
Minibatch loss: 0.063   Accuracy: 0.980
Minibatch loss: 0.141   Accuracy: 0.970
Minibatch loss: 0.062   Accuracy: 1.000
Minibatch loss: 0.049   Accuracy: 0.990
Minibatch loss: 0.059   Accuracy: 0.990
Minibatch loss: 0.082   Accuracy: 0.970

Epoch: 3
Minibatch loss: 0.063   Accuracy: 0.980
Minibatch loss: 0.085   Accuracy: 0.960
Minibatch loss: 0.084   Accuracy: 0.980
Minibatch loss: 0.080   Accuracy: 0.980
Minibatch loss: 0.018   Accuracy: 1.000
Minibatch loss: 0.022   Accuracy: 1.000
